# Pre-processing

In [30]:
import sys, os, glob, shutil
sys.path.append('../')

from MRIsegm.utils import get_slice, get_slice_info, get_rois, explore_slice, explore_roi, make_mask, mask_slice

from PIL import Image
from tqdm.notebook import trange
from time import sleep
import cv2

## Loading Data

In [31]:
src = '/Users/giuseppefilitto/Pazienti_anonym_sorted'

patiences = os.listdir(src)
if '.DS_Store' in patiences:
    patiences.remove('.DS_Store')

bad_patiences = [ 'BO9', 'BO17', 'BO28', 'BO36', 'BO37', 'BO39', 'BO40', 'BO54', 'BO72', 'BO77', 'BO86']

good_patiences = list(set(patiences) - set(bad_patiences))

# ! Removing for now 
good_patiences.remove('BO38')
print("Number of good patiences:",len(good_patiences))

Number of good patiences: 36


## Saving Train and Test images

### Training

In [32]:
progress_bar = trange(len(good_patiences), desc="Progress")
for patience in good_patiences:

    #! slices
    folder = 'T2'
    slice_path = os.path.join(src, patience, folder)

    if not os.path.isdir(slice_path):
        slice_path =  slice_path + "AX"

        if not os.path.isdir(slice_path):
            slice_path = os.path.join(src, patience, 'T25mm')

            if not os.path.isdir(slice_path):
                slice_path = os.path.join(src, patience, 't2DEF')


    slice = get_slice(dir_path=slice_path)  
    

    roi_folder = 'T2ROI'
    roi_path = os.path.join(src, patience, roi_folder)

    #! ROIS
    roi = get_rois(roi_path=roi_path)
    

    #! slices of masks

    slice_of_masks = mask_slice(slice=slice, rois=roi)
    

    positions = [roi[j].get('position') - 1 for j in range(len(roi))]

    
    for i in set(positions):

        img = Image.fromarray(slice[i, : ,:])
        filename = patience + '_slice_' + str(i) + '.png'
        dst = '../data/train/slices'
        output =  os.path.join(dst, filename)

        if os.path.isfile(output):
            os.remove(output)
        
        img.save(output)

        mask = Image.fromarray(slice_of_masks[i, : ,:])
        filename = patience + '_mask_' + str(i) + '.png'
        dst = '../data/train/masks'
        output =  os.path.join(dst, filename)
        if os.path.isfile(output):
            os.remove(output)

        mask.save(output)

        
           
    sleep(0.01)
    progress_bar.update(1)    

progress_bar.close()    

Progress:   0%|          | 0/36 [00:00<?, ?it/s]

### Test

In [33]:
train_dir = '../data/train/slices'
train_files = glob.glob(train_dir + '/*.png')

mask_dir = '../data/train/masks'
mask_files = glob.glob(mask_dir + '/*.png')

print("Training images", len(train_files))
# ! test

test_dir = '../data/test/slices'
ground_truth_dir = '../data/test/ground truth'

n_test = int(len(train_files) * 0.2)
print("Test images", n_test)

Training images 469
Test images 93


In [34]:
for i in range(0,n_test + 1):
    
    dir, file = os.path.split(train_files[i])
    test_path = os.path.join(test_dir, file)

    img = train_files[i]
    shutil.move(img, test_path)

    mask_dir = dir.replace('slices', 'masks')
    mask_file = file.replace('_slice_','_mask_')
    
    mask = os.path.join(mask_dir, mask_file)
    gt_path = os.path.join(ground_truth_dir, file)
    shutil.move(mask, gt_path)